In [1]:
import csv
from dataclasses import dataclass

@dataclass
class LogEntry:
  time: float
  source: str

# TODO: token parsing
@dataclass
class ATCommand(LogEntry):
  command: str

@dataclass
class BulkData(LogEntry):
  data: bytearray
  end: bool


def parse_log_entry(row):
  time = float(row['Time'])
  source = 'host' if row['Source'] == 'host' else 'device'
  data_length = int(row['Data length [bytes]'])
  leftover_data = row['Leftover Capture Data']
  at_stream = row['AT Stream'].replace('\\\\r', '\r')
  if at_stream:
    if data_length != len(at_stream):
      raise ValueError('Data length does not match AT Stream length', at_stream)
    if not at_stream.endswith('\r'):
      raise ValueError('AT Stream does not end with return character')
    if not at_stream.startswith('AT'):
      raise ValueError('AT Stream does not start with "AT"')
    return ATCommand(time, source, at_stream[2:-1])
  elif leftover_data:
    leftover_data = bytearray.fromhex(leftover_data)
    if data_length != len(leftover_data):
      raise ValueError('Data length does not match Leftover Capture Data length')
    end = leftover_data[-1] == 0x3e
    if end:
      leftover_data = leftover_data[:-1]
    return BulkData(time, source, leftover_data, end)
  else:
    raise ValueError('No data in log entry')


def parse_log(filename):
  with open(filename, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      yield parse_log_entry(row)

In [2]:
for m in parse_log('forscan_elm327_pscm_toggle_lca.csv'):
  direction = '>' if m.source == 'host' else '<'
  if isinstance(m, ATCommand):
    print(direction, f'AT{m.command}')
  elif isinstance(m, BulkData):
    print(direction, m.data.decode())
    if m.end:
      print()

> ATRV
< 14.4V

> ATRV
< 14.4V

> ATRV
< 14.4V

> ATSH000730
< OK

> ATCRA738
< OK

> 22DE00
< CAN ERROR

> 22DE00
1:8CDE00FF000A<RX ERROR
< 02000250003D
< 2:4D0DCD00000000
< 3:00000000000000
> s
< STOPPED

> 22DE00
0:62DE00FF000A
< 1:8C02000250003D
< 2:4D0DCD00000000
< 3:00000000000000
> s
< STOPPED

> 22DE01
1:FFFFFFFFFFFF
< F00
< 2:FF000000000000
< 3:00000000000000
> s
< STOPPED

> 22DE02
0:62DE02FFFF00
< 1:000200FF000000
< 2:00000000000000
< 3:00000000000000
> s
< STOPPED

> 22DE03
0:62DE03000000
2:0000000000000000
< 0
< 3:00000000000000
> s
< STOPPED

> 22DE04
0:62DE04000000
2:0000000000000000
< 0
< 3:00000000000000
> s
< STOPPED

> 22DE05
< 7F2231
> s
< STOPPED

> 22DE06
< 7F2231
> s
< STOPPED

> 22DE07
< 7F2231
> s
< STOPPED

> 22DE08
< 7F2231
> s
< STOPPED

> 22DE09
< 7F2231
> s
< STOPPED

> 22DE0A
< 7F2231
> s
< STOPPED

> 22DE0B
< 7F2231
> s
< STOPPED

> 22DE0C
< 7F2231
> s
< STOPPED

> 22DE0D
< 7F2231
> s
< STOPPED

> 22DE0E
< 7F2231
> s
< STOPPED

> ATCAF0
< OK

> 100F22000